In [ ]:
pip install ydata-profiling

In [ ]:
import pandas as pd
import json
from ydata_profiling import ProfileReport

In [ ]:
df = pd.read_json('dataset_big_patent_v3.json')
profile = ProfileReport(df, title="Profiling Report")

In [ ]:
profile.to_file("report.html")

In [ ]:
profile.to_notebook_iframe()

In [ ]:
with open('dataset_big_patent_v3.json') as f:
    data = json.load(f)

df = pd.json_normalize(data, meta=['anchor', "query", "positive", "negative",])

# Display the DataFrame
df

In [ ]:
df.isna().sum()

In [ ]:
from IPython.display import display, HTML

display(HTML(df.sample(n=10).to_html()))

In [ ]:
pip install transformers

In [ ]:
pip install torch

In [ ]:
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

# text = "The present disclosure relates to a disposable loading unit (DLU) or single use loading unit (SULU) for use with a surgical device"

# tokens = tokenizer(text, 
#                    padding=True, # Ajoute du padding pour uniformiser la longueur
#                    truncation=True, # Tronque les séquences trop longues
#                    return_tensors="pt") # Retourne des tenseurs PyTorch

# print(tokens)

In [ ]:
from sklearn.model_selection import train_test_split

test_size = 0.2
val_size = 0.125

train_df, test_df = train_test_split(df, 
# 80% train, 20% test                              test_size=test_size,
                                     random_state=39)
train_df, val_df = train_test_split(train_df, 
                                     test_size=val_size/(1-test_size),
                                     random_state=39)
# 70% train, 10% validation

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

def tokenize_function(examples):
    return tokenizer(examples["abstract"], truncation=True, padding="max_length", max_length=max_length, return_tensors="pt")

In [ ]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, dataframe, texts, tokenizer, max_length):
        self.texts = []
        for column in column_names:
            self.texts.extend(dataframe[column].astype(str).tolist())  # Ajoute le texte de chaque colonne à la liste
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer(text,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=self.max_length,
                                   return_tensors='pt')
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': encoding['input_ids'].flatten()  # Pour l'auto-encoding
        }

In [ ]:
# Nom des colonnes à tokeniser
column_names = df.columns.tolist()

In [ ]:
train_dataset = CustomDataset(train_df, tokenizer, 512, column_names)
val_dataset = CustomDataset(val_df, tokenizer, 512, column_names)
test_dataset = CustomDataset(test_df, tokenizer, 512, column_names)

In [ ]:
pip install transformers[torch]

In [ ]:
# from transformers import AutoModel, AutoTokenizer, TrainingArguments, Trainer

# training_args = TrainingArguments(
#     output_dir="./results",          # Dossier pour sauvegarder les résultats
#     overwrite_output_dir=True,
#     num_train_epochs=3,              # Nombre d'epochs
#     per_device_train_batch_size=16,  # Taille du batch par GPU
#     per_device_eval_batch_size=64,   # Taille du batch pour l'évaluation
#     save_steps=1000,                # Fréquence de sauvegarde des checkpoints
#     save_total_limit=2,              # Nombre maximum de checkpoints à conserver
#     prediction_loss_only=True,       # Optimise seulement la loss de prédiction
#     eval_steps=500,                  # Fréquence d'évaluation
#     learning_rate=5e-5,             # Taux d'apprentissage
#     weight_decay=0.01,               # Décroissance du poids
# )

# # 5. Entraînement du modèle
# model = AutoModel.from_pretrained("BAAI/bge-m3")

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
# )

# trainer.train()

# # 6. Sauvegarde du modèle fine-tuned
# model.save_pretrained("./bge-m3-fine-tuned")
# tokenizer.save_pretrained("./bge-m3-fine-tuned")

# print("Entraînement terminé !")

In [ ]:
# from transformers import AutoModel, AutoTokenizer, TrainingArguments, Trainer
# import torch
# from torch.utils.data import Dataset
# import pandas as pd

# # 1. Préparation des données
# class CustomDataset(Dataset):
#     def __init__(self, texts, tokenizer, max_length):
#         self.texts = texts
#         self.tokenizer = tokenizer
#         self.max_length = max_length

#     def __len__(self):
#         return len(self.texts)

#     def __getitem__(self, idx):
#         text = str(self.texts[idx])
#         encoding = self.tokenizer(text,
#                                    truncation=True,
#                                    padding='max_length',
#                                    max_length=self.max_length,
#                                    return_tensors='pt')
#         return {
#             'input_ids': encoding['input_ids'].flatten(),
#             'attention_mask': encoding['attention_mask'].flatten()
#         }

# # 2. Chargement du modèle et du tokenizer
# model_name = "BAAI/bge-m3"  # Remplace par le nom exact du modèle
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name)

# # 3. Préparation du dataset
# # Charge tes données à partir d'un fichier (CSV, JSON, etc.)
# data = pd.read_json('dataset_big_patent_v3.json')
# data = data.dropna(subset=['abstract'])
# texts = data['abstract'].tolist()  # Remplace 'text_column' par le nom de ta colonne de texte
# max_length = 512  # Adapte la longueur maximale des séquences

# dataset = CustomDataset(texts, tokenizer, max_length)

# # 4. Configuration de l'entraînement
# training_args = TrainingArguments(
#     output_dir="./results",          # Dossier pour sauvegarder les résultats
#     overwrite_output_dir=True,
#     num_train_epochs=3,              # Nombre d'epochs
#     per_device_train_batch_size=16,  # Taille du batch par GPU
#     save_steps=1000,                # Fréquence de sauvegarde des checkpoints
#     save_total_limit=2,              # Nombre maximum de checkpoints à conserver
#     prediction_loss_only=True,       # Optimise seulement la loss de prédiction
#     learning_rate=5e-5,             # Taux d'apprentissage
#     weight_decay=0.01,               # Décroissance du poids
# )

# # 5. Entraînement du modèle
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=dataset,
# )

# trainer.train()

# # 6. Sauvegarde du modèle fine-tuned
# model.save_pretrained("./bge-m3-fine-tuned")
# tokenizer.save_pretrained("./bge-m3-fine-tuned")

# print("Entraînement terminé !")


In [ ]:
# Nouvelle piste
# https://dassum.medium.com/fine-tune-large-language-model-llm-on-a-custom-dataset-with-qlora-fb60abdeba07

In [ ]:
pip install -q -U bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl rouge_score

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
from tqdm import tqdm
from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np

In [ ]:
import os
# disable Weights and Biases
os.environ['WANDB_DISABLED']="true"

In [ ]:
# 1. Loading datatset
df = pd.read_json('dataset_big_patent_v3.json')

In [ ]:
# 2. Create Bitsandbytes configuration
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )

In [ ]:
pip install -U bitsandbytes

In [ ]:
# 3. Loading the Pre-Trained model
from transformers import AutoTokenizer

model_name = "intfloat/e5-mistral-7b-instruct"

device_map = {"": 0}
original_model = AutoModelForCausalLM.from_pretrained(model_name, 
                                                      device_map=device_map,
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      token=True)



In [ ]:
# 4. Tokenization with left padding

tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True,padding_side="left",add_eos_token=True,add_bos_token=True,use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# 5. Test the model with zero-shopt inferencing
%%time
from transformers import set_seed
seed = 42
set_seed(seed)

index = 10

prompt = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

formatted_prompt = f"Instruct: Summarize the following conversation.\n{prompt}\nOutput:\n"
res = gen(original_model,formatted_prompt,100,)
#print(res[0])
output = res[0].split('Output:\n')[1]

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{formatted_prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

# Choses à faire demain

1. jupyter notebook gpu acceleration -> Create venv Skipped
2. https://www.datacamp.com/fr/tutorial/fine-tuning-large-language-models
3. Savoir quelle colonne tokenizer
4. Supervised learning -> Q&A # https://towardsdatascience.com/fine-tuning-large-language-models-llms-23473d763b91/

In [ ]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct-GPTQ-Int4"

In [ ]:
# 1. Loading datatset
import pandas as pd

df = pd.read_json('dataset_big_patent_v3.json')

In [ ]:
from sklearn.model_selection import train_test_split

test_size = 0.2
val_size = 0.125

train_df, test_df = train_test_split(df, 
# 80% train, 20% test                              test_size=test_size,
                                     random_state=39)
train_df, val_df = train_test_split(train_df, 
                                     test_size=val_size/(1-test_size),
                                     random_state=39)
# 70% train, 10% validation

train_df = train_df.copy()
val_df = val_df.copy()
test_df = test_df.copy()

train_df['split'] = 'train'
val_df['split'] = 'val'
test_df['split'] = 'test'

df_final = pd.concat([train_df, val_df, test_df]).reset_index(drop=True)

df_final

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset, Dataset, DatasetDict

dataset = load_dataset("json", data_files="dataset_big_patent_v3.json")

from sklearn.model_selection import train_test_split

test_size = 0.2
val_size = 0.125

train_df, test_df = train_test_split(df,
                                     test_size=test_size,
                                     random_state=39)

ds_train = Dataset.from_pandas(train_df)
ds_test = Dataset.from_pandas(test_df)

dataset_dict = DatasetDict({
    "train": ds_train,
    "test": ds_test
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(examples["anchor"], truncation=True, padding="max_length")

tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42)
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42)

In [ ]:
!pip install tf-keras

In [ ]:
import torch
print(torch.__version__)

In [ ]:
import torch
print("CUDA disponible :", torch.cuda.is_available())
print("Nombre de GPU :", torch.cuda.device_count())
if torch.cuda.is_available():
    print("Nom du GPU :", torch.cuda.get_device_name(0))
    print("Version CUDA utilisée par PyTorch :", torch.version.cuda)

In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quantization_config
)

In [ ]:
# from transformers import AutoModel, AutoTokenizer, TrainingArguments, Trainer

# original_model = AutoModel.from_pretrained(model_name, num_labels=3)

In [ ]:
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   return metric.compute(predictions=predictions, references=labels)

In [ ]:
# import tensorflow as tf

# print(tf.config.list_physical_devices('GPU'))

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.version.cuda)

In [ ]:
import logging

# Niveau de log : DEBUG pour tout voir
logging.basicConfig(level=logging.DEBUG)

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:64'
import torch

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

In [ ]:
!nvidia-smi

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
   output_dir="test_trainer",
   #evaluation_strategy="epoch",
   per_device_train_batch_size=1,  # Reduce batch size here
   per_device_eval_batch_size=1,    # Optionally, reduce for evaluation as well
   gradient_accumulation_steps=4
   )


trainer = Trainer(
   model=original_model,
   args=training_args,
   train_dataset=small_train_dataset,
   eval_dataset=small_eval_dataset,
   compute_metrics=compute_metrics,

)

trainer.train()


In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [62]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset, Dataset, DatasetDict

dataset = load_dataset("json", data_files="dataset_big_patent_v3.json", split="train")

bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4")
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-0.5B-Instruct",
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")

In [63]:
import pandas as pd

pd.DataFrame(dataset)

,anchor,query,positive,negative
0,RELATED APPLICATIONS This application claims t...,What are the key advantages and applications o...,The present technology introduces an innovativ...,The invention relates to the design and utilit...
1,RELATED APPLICATIONS This application claims t...,How does a magnetic energy harvester operate w...,The advanced energy accumulation equipment bei...,The invention relates to the design and utilit...
2,RELATED APPLICATIONS This application claims t...,How does an energy harvester operate without a...,The invention relates to the design and utilit...,The present technology introduces an innovativ...
3,BACKGROUND OF THE INVENTION I. Field of the In...,How can buffer blocks for ruminant animals be ...,The innovative technique pertains to mineral s...,The latest invention provides novel systems an...
4,RELATED APPLICATION The present application cl...,What advancements does the described patent pr...,The current text discusses a novel mechanical ...,The present invention addresses various improv...
...,...,...,...,...
494,CROSS-REFERENCE TO RELATED APPLICATION [0001] ...,What is the role of Onjisaponin B in the treat...,Delineation of the effect of a botanical enhan...,This invention pertains to a unique compound f...
495,CROSS-REFERENCE TO RELATED APPLICATION [0001] ...,What is the mechanism by which Onjisaponin B e...,This invention pertains to a unique compound f...,Delineation of the effect of a botanical enhan...
496,RELATED APPLICATIONS [0001] This application i...,What are the advantages of the improved dental...,The present innovations focus on an enhanced s...,[0001] This document covers the detailed aspec...
497,FIELD [0001] The invention refers to a ventric...,What are the benefits and mechanisms of a nove...,The technology relates to an innovation in hem...,The present technology involves an advancement...


In [64]:
def preprocess_function(dataset):
    texts = [f"Context: {c}\nQuestion: {q}\nAnswer: {a}" for c, q, a in zip(dataset["anchor"], dataset["query"], dataset["positive"])]
    # Tokenize
    model_inputs = tokenizer(texts, max_length=384, truncation=True, padding="max_length")
    # labels = input_ids
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

In [65]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [66]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./qwen-qa-finetune",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4, # TO TEST
    fp16=True,
    save_strategy="epoch",
    logging_steps=10,
)

In [67]:
# https://www.learnpytorch.io/pytorch_cheatsheet/
# Setup device-agnostic code 
if torch.cuda.is_available():
    device = "cuda" # NVIDIA GPU
elif torch.backends.mps.is_available():
    device = "mps" # Apple GPU
else:
    device = "cpu" # Defaults to CPU if NVIDIA GPU/Apple GPU aren't available

In [68]:
# Zero-shot performance

prompt = "Question: How does the crowdsourcing method is used to adjust a video game element ?\nAnswer:" # Expected : A processor retrieves a plurality of received game element feedback data from a plurality of users of a game and causes the game element to be adjusted during execution of the game 
inputs = tokenizer(prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Question: How does the crowdsourcing method is used to adjust a video game element ?
Answer: The crowdsourced method uses the data of other users and then creates a new version of the game. This can be done in several ways, including creating a new level that will have more interesting gameplay elements, or by adding new levels to an existing one


In [69]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],  # adapte selon le modèle
)
model = get_peft_model(model, lora_config)

In [70]:
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)
trainer.train()


C:\Users\Matts\AppData\Local\Temp\ipykernel_15304\3724420681.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,2.353800
20,2.160700
30,2.123100
40,2.180800
50,2.109600
60,2.146200
70,2.024300
80,2.244300
90,1.957800
100,2.069100


In [ ]:
import evaluate

trainer.evaluate()

In [ ]:
# Fine-tuned performance after

prompt = "Question: How does the crowdsourcing method is used to adjust a video game element ?\nAnswer:" # Expected : A processor retrieves a plurality of received game element feedback data from a plurality of users of a game and causes the game element to be adjusted during execution of the game
inputs = tokenizer(prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

# Conclusion et prochaines étapes

1. Compare la performance zero-shot vs fine-tuning
2. Si peu ou pas d’amélioration, il faut peut-être :

Augmenter la taille ou la qualité du dataset.

Ajuster le prompt pour le zero-shot.

Continuer le fine-tuning avec plus d’époques ou de paramètres.

---

Prochaines étapes :
Améliorer la qualité des prompts (prompt engineering) pour le zero-shot.

Augmenter la taille ou la diversité du dataset pour le fine-tuning.

Expérimenter avec différents hyperparamètres (learning rate, batch size, nombre d’époques).

Data augmentation pour augmenter la variété.

Validation croisée pour une meilleure généralisation